# 1. kakaocorp/kanana-1.5-8b-instruct-2505 단순 모델 불러오기

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "kakaocorp/kanana-1.5-8b-instruct-2505"

tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.eval()

def _build_inputs(messages):
    return tok.apply_chat_template(
          messages,
          add_generation_prompt=True,
          return_tensors="pt"
      ).to(model.device)

def chat(user_msg,
         system_msg="너는 보이스피싱 탐지기야."
                    "실제 공공기관의 정상 절차와 비교해 이상한 점을 설명하고, "
                    "어떤 보이스피싱 수법인지 2-3문장으로 규정해라. 한국어만 사용."):
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg},
    ]
    inputs = _build_inputs(messages)

    full_prompt = tok.decode(inputs[0], skip_special_tokens=False)
    print("--- 전체 프롬프트 확인 ---")
    print(full_prompt)
    print("------------------------")

    with torch.no_grad():
        out = model.generate(
            inputs,
            max_new_tokens=225,
            do_sample=True, temperature=0.7, top_p=0.9,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )
    print(tok.decode(out[0][inputs.shape[1]:], skip_special_tokens=True))

case_text = "본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?"
chat(case_text)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

--- 전체 프롬프트 확인 ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>


<|eot_id|><|start_header_id|>system<|end_header_id|>

너는 보이스피싱 탐지기야.실제 공공기관의 정상 절차와 비교해 이상한 점을 설명하고, 어떤 보이스피싱 수법인지 2-3문장으로 규정해라. 한국어만 사용.<|eot_id|><|start_header_id|>user<|end_header_id|>

본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?<|eot_id|><|start_header_id|>assistant<|end_header_id|>


------------------------
이 문장은 실제 공공기관이나 수사기관에서 사용하는 정상 절차와 여러 가지로 다릅니다.

첫째, 피해자가 법적 조치나 절차에 참여할 수 없다는 주장은 공공기관이나 수사기관이 절대적으로 금하는 내용입니다. 정상적으로는 피해자에게 필요한 안내와 권리 보장을 해줍니다.

둘째, “혐의자 신분”이라는 표현이나 “엠바고 사건” 등의 용어 사용은 공공기관의 공식 절차와 다르며, 불안과 압박을 조성해 개인정보나 추가 정보를 요구하는 전형적인 보이스피싱 수법입니다.

이런 내용은 보이스피싱 범죄자가 피해자를 혼란스럽게 하여 개인정보나 돈을 요구하는 수법입니다.


In [ ]:
case_text = "만약 당신이 정말로 구속수사는 면하고 싶다 그러시면 본인 담당검사 있죠? 메모하세요. 신원보증 서류, 개인의견서 이 두가지 서류를 금융감독원으로 보내달라고 하시고 제일 중요한 거는 담당 검사가 당신을 신원보증 서주는 게 제일 중요합니다. 만약 담당 검사가 본인을 신원보증 서주겠다고 그러면 제가 약식명령으로 생각을 다시 한번 해볼께요. 근데 아마 담당 검사가 그럴 일은 없겠죠? 혐의자 신분인 본 인한테 뭘 믿고 보증을 서주겠어요? 일단 담당검사한테 울고불고 부탁을 해서라도 신원보증 받는 게 제 일 중요합니다."
chat(case_text)

--- 전체 프롬프트 확인 ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>


<|eot_id|><|start_header_id|>system<|end_header_id|>

너는 보이스피싱 탐지기야.실제 공공기관의 정상 절차와 비교해 이상한 점을 설명하고, 어떤 보이스피싱 수법인지 2-3문장으로 규정해라. 한국어만 사용.<|eot_id|><|start_header_id|>user<|end_header_id|>

만약 당신이 정말로 구속수사는 면하고 싶다 그러시면 본인 담당검사 있죠? 메모하세요. 신원보증 서류, 개인의견서 이 두가지 서류를 금융감독원으로 보내달라고 하시고 제일 중요한 거는 담당 검사가 당신을 신원보증 서주는 게 제일 중요합니다. 만약 담당 검사가 본인을 신원보증 서주겠다고 그러면 제가 약식명령으로 생각을 다시 한번 해볼께요. 근데 아마 담당 검사가 그럴 일은 없겠죠? 혐의자 신분인 본 인한테 뭘 믿고 보증을 서주겠어요? 일단 담당검사한테 울고불고 부탁을 해서라도 신원보증 받는 게 제 일 중요합니다.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


------------------------
이 전화는 보이스피싱 수법입니다.

이 수법은 실제 공공기관이나 수사기관(검찰 등)에서는 개인의견서나 신원보증서류를 금융감독원 등 외부기관으로 직접 제출하라고 요구하지 않습니다. 또한, 범죄 혐의자에게 담당 검사가 개인적으로 신원보증을 서주겠다고 하거나, 이런 서류 제출을 요구하는 것은 정상 절차가 아니므로 주의해야 합니다.


# 2. RAG 구축
- 문서로드
- 청크분할
- 임베딩
- 벡터저장 & 검색기 생성
- LLM 연결

### ⓐ 문서로드

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install langchain_community faiss-gpu-cu12

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; p

In [1]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/drive/MyDrive/arim/document.txt", encoding="utf-8")
documents = loader.load()

In [ ]:
documents

[Document(metadata={'source': '/content/drive/MyDrive/arim/document.txt'}, page_content='# 문서1: 국민은행 보이스피싱 10계명\n① 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심\n검찰∙경찰∙금감원 등 정부기관은 어떠한 경우에도 전화로 자금의 이체 또는 개인의 금융거래정보를 요구하지 않습니다. 정부기관을  사칭, 범죄에 연루되었다며 금융거래 정보를 요구하거나 안전조치 등을 명목으로 자금의 이체 등을 요구하는 경우는 100% 보이스피싱이므로 이러한 전화를 받는 경우 전화를 끊고 해당 기관의 대표전화로 전화하여 사실여부를 반드시 확인하시기 바랍니다.\n② 대출 처리비용 등을 이유로 선입금 요구시 보이스피싱을 의심\n정상적인 금융회사는 전산비용, 보증료, 저금리 전환 예치금, 선이자 등 어떠한 명목으로도 대출과 관련하여 선입금하라고 요구하지 않으므로, 이러한 요구에 절대로 응해서는 안됩니다.\n③ 출처 불명 파일∙이메일∙문자는 클릭하지 말고 삭제\n출처가 불분명한 파일을 다운받거나 의심스러운 인터넷 주소가 포함된 문자를 클릭하면 악성 코드에 감염되어 개인정보가 유출될 수 있습니다. 악성코드 감염은 금융거래 시 파밍 등을 일으키는 주요 원인이므로 이러한 파일이나 문자는 즉시 삭제하시기 바랍니다.\n④  금감원 팝업창 뜨고 금융거래정보 입력 요구시 100% 보이스피싱\n인터넷 포털사이트에 접속시, 보안관련 인증절차를 진행한다는 내용의 금감원 팝업창이 뜨며, 이를  클릭하면 보안승급을 위해서라며 계좌번호, 비밀번호, 보안카드번호 등 금융거래정보를 입력하라고 요구하면 보이스피싱(파밍)이니 절대 응해서는 안됩니다.\n⑤ 고금리 대출 먼저 받아 상환하면 신용등급이 올라 저금리 대출이 가능하다는 말은 보이스피싱\n정상적인 금융회사는 저금리 대출을 받기 위해서 고금리 대출을 먼저 받으라고 요구하지 않습니다. 저금리 대출을 받기 위해서는 거래실적을 쌓아야 한다며 고금리대출을 먼저 받으라고 하는

### ⓑ 청크분할

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

'''
문서를 분할할 기준(구분자)을 우선순위 순으로 지정
 - '---'와 '# 문서'는 큰 문서 단위
 - ①, ② 등은 세부 내용 단위
chunk_size=청크의 최대 크기
chunk_overlap=청크 간 겹치는 부분 (맥락 유지를 위함)
'''

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n---\n", "①", "②", "③", "④", "⑤", "\n\n", "\n"],
    chunk_size=1000,
    chunk_overlap=100
)

split_documents = text_splitter.split_documents(documents)

In [ ]:
split_documents

[Document(metadata={'source': 'document.txt'}, page_content='# 문서1: 국민은행 보이스피싱 10계명\n---\n① 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심\n검찰∙경찰∙금감원 등 정부기관은 어떠한 경우에도 전화로 자금의 이체 또는 개인의 금융거래정보를 요구하지 않습니다. 정부기관을  사칭, 범죄에 연루되었다며 금융거래 정보를 요구하거나 안전조치 등을 명목으로 자금의 이체 등을 요구하는 경우는 100% 보이스피싱이므로 이러한 전화를 받는 경우 전화를 끊고 해당 기관의 대표전화로 전화하여 사실여부를 반드시 확인하시기 바랍니다.\n② 대출 처리비용 등을 이유로 선입금 요구시 보이스피싱을 의심\n정상적인 금융회사는 전산비용, 보증료, 저금리 전환 예치금, 선이자 등 어떠한 명목으로도 대출과 관련하여 선입금하라고 요구하지 않으므로, 이러한 요구에 절대로 응해서는 안됩니다.\n③ 출처 불명 파일∙이메일∙문자는 클릭하지 말고 삭제\n출처가 불분명한 파일을 다운받거나 의심스러운 인터넷 주소가 포함된 문자를 클릭하면 악성 코드에 감염되어 개인정보가 유출될 수 있습니다. 악성코드 감염은 금융거래 시 파밍 등을 일으키는 주요 원인이므로 이러한 파일이나 문자는 즉시 삭제하시기 바랍니다.\n④  금감원 팝업창 뜨고 금융거래정보 입력 요구시 100% 보이스피싱\n인터넷 포털사이트에 접속시, 보안관련 인증절차를 진행한다는 내용의 금감원 팝업창이 뜨며, 이를  클릭하면 보안승급을 위해서라며 계좌번호, 비밀번호, 보안카드번호 등 금융거래정보를 입력하라고 요구하면 보이스피싱(파밍)이니 절대 응해서는 안됩니다.\n⑤ 고금리 대출 먼저 받아 상환하면 신용등급이 올라 저금리 대출이 가능하다는 말은 보이스피싱\n정상적인 금융회사는 저금리 대출을 받기 위해서 고금리 대출을 먼저 받으라고 요구하지 않습니다. 저금리 대출을 받기 위해서는 거래실적을 쌓아야 한다며 고금리대출을 먼저 받으라고 하는 경우는 100% 보이스피싱입니다. 또한 

---------------------------------------------
 원하는 Chunk 단위가 아님
---------------------------------------------

In [2]:
import re, unicodedata
from langchain_community.document_loaders import TextLoader
from langchain.schema import Document

FILE = "/content/drive/MyDrive/arim/document.txt"

SRC = {1: "KB_10", 2: "헌법제12조", 3: "약식명령", 4: "형사소송법"}
PFX = {1: "KB",    2: "CONST12",   3: "YAK",    4: "CPL"}

# ①~⑳ 보존: NFKC 금지, NFC만 사용
text = unicodedata.normalize("NFC", TextLoader(FILE, encoding="utf-8").load()[0].page_content)

# 1) '# 문서N: 제목'으로 1차 분할
hdr_re = re.compile(r'^#\s*문서\s*(\d+)\s*[:：]\s*(.+)$', re.M)
headers = list(hdr_re.finditer(text))

docs = []
for i, h in enumerate(headers):
    num   = int(h.group(1))
    title = h.group(2).strip()
    a = h.end()
    b = headers[i+1].start() if i+1 < len(headers) else len(text)
    body = text[a:b].strip()

    # 2) 라인 시작의 원형숫자(①~⑳)를 섹션 헤더로 보고 블록 자르기
    head_re = re.compile(r'(?m)^[ \t]*([①-⑳])\s*(.+)$')
    heads = list(head_re.finditer(body))

    if not heads:  # 원형숫자 없으면 본문 전체 1청크
        docs.append(Document(
            page_content=body,
            metadata={"source": SRC.get(num, f"문서{num}"),
                      "section_id": f"{PFX.get(num, f'DOC{num}')}-본문",
                      "title": title}
        ))
        continue

    for j, m in enumerate(heads):
        s = m.start()
        e = heads[j+1].start() if j+1 < len(heads) else len(body)
        chunk = body[s:e].strip()                     # ⟵ 소제목 + 그 아래 본문 전체
        circ  = m.group(1)                            # ①, ②, …
        sub   = m.group(2).strip()                    # 소제목 텍스트
        docs.append(Document(
            page_content=chunk,
            metadata={"source": SRC.get(num, f"문서{num}"),
                      "section_id": f"{PFX.get(num, f'DOC{num}')}-{circ}",
                      "title": sub or title}
        ))

# ====== 점검 출력 ======
print("총 청크:", len(docs))
for k in ("KB_10","헌법제12조","약식명령","형사소송법"):
    print(f"{k}: ", sum(1 for d in docs if d.metadata['source']==k), "개")

for i, d in enumerate(docs, 1):
    print(f"\n[청크{i}] {d.metadata['section_id']} | {d.metadata['title']}\n{d.page_content}")

총 청크: 18
KB_10:  5 개
헌법제12조:  7 개
약식명령:  5 개
형사소송법:  1 개

[청크1] KB-① | 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심
① 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심
검찰∙경찰∙금감원 등 정부기관은 어떠한 경우에도 전화로 자금의 이체 또는 개인의 금융거래정보를 요구하지 않습니다. 정부기관을  사칭, 범죄에 연루되었다며 금융거래 정보를 요구하거나 안전조치 등을 명목으로 자금의 이체 등을 요구하는 경우는 100% 보이스피싱이므로 이러한 전화를 받는 경우 전화를 끊고 해당 기관의 대표전화로 전화하여 사실여부를 반드시 확인하시기 바랍니다.

[청크2] KB-② | 대출 처리비용 등을 이유로 선입금 요구시 보이스피싱을 의심
② 대출 처리비용 등을 이유로 선입금 요구시 보이스피싱을 의심
정상적인 금융회사는 전산비용, 보증료, 저금리 전환 예치금, 선이자 등 어떠한 명목으로도 대출과 관련하여 선입금하라고 요구하지 않으므로, 이러한 요구에 절대로 응해서는 안됩니다.

[청크3] KB-③ | 출처 불명 파일∙이메일∙문자는 클릭하지 말고 삭제
③ 출처 불명 파일∙이메일∙문자는 클릭하지 말고 삭제
출처가 불분명한 파일을 다운받거나 의심스러운 인터넷 주소가 포함된 문자를 클릭하면 악성 코드에 감염되어 개인정보가 유출될 수 있습니다. 악성코드 감염은 금융거래 시 파밍 등을 일으키는 주요 원인이므로 이러한 파일이나 문자는 즉시 삭제하시기 바랍니다.

[청크4] KB-④ | 금감원 팝업창 뜨고 금융거래정보 입력 요구시 100% 보이스피싱
④  금감원 팝업창 뜨고 금융거래정보 입력 요구시 100% 보이스피싱
인터넷 포털사이트에 접속시, 보안관련 인증절차를 진행한다는 내용의 금감원 팝업창이 뜨며, 이를  클릭하면 보안승급을 위해서라며 계좌번호, 비밀번호, 보안카드번호 등 금융거래정보를 입력하라고 요구하면 보이스피싱(파밍)이니 절대 응해서는 안됩니다.

[청크5] KB-⑤ | 고금리 대출 먼저 받아 상환

In [ ]:
docs

[Document(metadata={'source': 'KB_10', 'section_id': 'KB-①', 'title': '전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심'}, page_content='① 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심\n검찰∙경찰∙금감원 등 정부기관은 어떠한 경우에도 전화로 자금의 이체 또는 개인의 금융거래정보를 요구하지 않습니다. 정부기관을  사칭, 범죄에 연루되었다며 금융거래 정보를 요구하거나 안전조치 등을 명목으로 자금의 이체 등을 요구하는 경우는 100% 보이스피싱이므로 이러한 전화를 받는 경우 전화를 끊고 해당 기관의 대표전화로 전화하여 사실여부를 반드시 확인하시기 바랍니다.'),
 Document(metadata={'source': 'KB_10', 'section_id': 'KB-②', 'title': '대출 처리비용 등을 이유로 선입금 요구시 보이스피싱을 의심'}, page_content='② 대출 처리비용 등을 이유로 선입금 요구시 보이스피싱을 의심\n정상적인 금융회사는 전산비용, 보증료, 저금리 전환 예치금, 선이자 등 어떠한 명목으로도 대출과 관련하여 선입금하라고 요구하지 않으므로, 이러한 요구에 절대로 응해서는 안됩니다.'),
 Document(metadata={'source': 'KB_10', 'section_id': 'KB-③', 'title': '출처 불명 파일∙이메일∙문자는 클릭하지 말고 삭제'}, page_content='③ 출처 불명 파일∙이메일∙문자는 클릭하지 말고 삭제\n출처가 불분명한 파일을 다운받거나 의심스러운 인터넷 주소가 포함된 문자를 클릭하면 악성 코드에 감염되어 개인정보가 유출될 수 있습니다. 악성코드 감염은 금융거래 시 파밍 등을 일으키는 주요 원인이므로 이러한 파일이나 문자는 즉시 삭제하시기 바랍니다.'),
 Document(metadata={'source': 'KB_10', 'section_id': 'KB-④', 'title': '금감원 팝업창

### ⓒ 임베딩
- 한국어 특화 임베딩모델 사용 (성능 ▼)
- 한국어+법률 특화 임베딩모델 사용

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="nlpai-lab/KURE-v1" # 한국어 법률 모델
)

/tmp/ipython-input-2053301124.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-MiniLM-L12-v2"  # 빠르고 가벼운 모델
)

/tmp/ipython-input-1596254501.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(docs, embeddings)

### ⓓ 검색기 생성

In [5]:
# 검색기(Retriever) 생성
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [6]:
suspect_message = "본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?"

relevant_docs = retriever.invoke(suspect_message) # retriever의 'invoke' 메서드를 사용하여 가장 관련성 높은 문서 검색

print("nlpai-lab/KURE-v1 임베딩 모델")
print(f"총 {len(relevant_docs)}개의 문서가 검색되었습니다.")
for i, doc in enumerate(relevant_docs):
    print(doc.page_content)

nlpai-lab/KURE-v1 임베딩 모델
총 3개의 문서가 검색되었습니다.
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.
⑤ 누구든지 체포 또는 구속의 이유와 변호인의 조력을 받을 권리가 있음을 고지받지 아니하고는 체포 또는 구속을 당하지 아니한다. 체포 또는 구속을 당한 자의 가족등 법률이 정하는 자에게는 그 이유와 일시ㆍ장소가 지체없이 통지되어야 한다.
③ 체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피 또는 증거인멸의 염려가 있을 때에는 사후에 영장을 청구할 수 있다.


In [ ]:
suspect_message = "본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?"

relevant_docs = retriever.invoke(suspect_message) # retriever의 'invoke' 메서드를 사용하여 가장 관련성 높은 문서 검색

print("paraphrase-multilingual-MiniLM-L12-v2 임베딩 모델")
print(f"총 {len(relevant_docs)}개의 문서가 검색되었습니다.")
for i, doc in enumerate(relevant_docs):
    print(doc.page_content)

paraphrase-multilingual-MiniLM-L12-v2 임베딩 모델
총 10개의 문서가 검색되었습니다.
② 검사의 약식명령 청구
검사는 폭행·상해사건이 벌금, 과료 또는 몰수의 형벌에 해당하여 공판절차 없이 약식명령으로 하는 것이 적당하다고 인정되는 경우에는 지방법원에 공소 제기와 동시에 서면으로 약식명령을 청구합니다(「형사소송법」 제448조제1항, 제449조 및 제450조).
약식명령을 청구 받은 지방법원은 해당 사건이 약식명령으로 할 수 없거나 약식명령으로 하는 것이 적당하지 않다고 인정되는 경우에는 공판절차에 따라 심판합니다(「형사소송법」 제450조).
① 약식명령의 개념
약식명령이란, 공판절차 없이 약식절차에 따라 벌금, 과료 또는 몰수 등 재산형을 과하는 지방법원의 재판을 말하며, 약식명령 시에는 추징, 그 밖의 부수의 처분도 할 수 있습니다(법령용어사례집, 법제처·한국법제연구원, 2003).
④ 약식명령의 고지
약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지됩니다(「형사소송법」 제452조).
제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.
② 모든 국민은 고문을 받지 아니하며, 형사상 자기에게 불리한 진술을 강요당하지 아니한다.
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.
③ 체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피 또는 증거인멸의 염려가 있을 때에는 사후에 영장을 청구할 수 있다.
⑤ 누구든지 체포 또는 구속의 이유와 변호인의 조력을 받을 권리가 있음을 고지받지 아니하고는 체포 또는 구속을 당하지 아니한다. 체포 또는 구속을 당한 자

In [ ]:
suspect_message = "만약 당신이 정말로 구속수사는 면하고 싶다 그러시면 본인 담당검사 있죠? 메모하세요. 신원보증 서류, 개인의견서 이 두가지 서류를 금융감독원으로 보내달라고 하시고 제일 중요한 거는 담당 검사가 당신을 신원보증 서주는 게 제일 중요합니다. 만약 담당 검사가 본인을 신원보증 서주겠다고 그러면 제가 약식명령으로 생각을 다시 한번 해볼께요. 근데 아마 담당 검사가 그럴 일은 없겠죠? 혐의자 신분인 본 인한테 뭘 믿고 보증을 서주겠어요? 일단 담당검사한테 울고불고 부탁을 해서라도 신원보증 받는 게 제 일 중요합니다."

relevant_docs = retriever.invoke(suspect_message)

print("nlpai-lab/KURE-v1 임베딩 모델")
print(f"총 {len(relevant_docs)}개의 문서가 검색되었습니다.")
for i, doc in enumerate(relevant_docs):
    print(doc.page_content)

nlpai-lab/KURE-v1 임베딩 모델
총 10개의 문서가 검색되었습니다.
③ 체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피 또는 증거인멸의 염려가 있을 때에는 사후에 영장을 청구할 수 있다.
① 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심
검찰∙경찰∙금감원 등 정부기관은 어떠한 경우에도 전화로 자금의 이체 또는 개인의 금융거래정보를 요구하지 않습니다. 정부기관을  사칭, 범죄에 연루되었다며 금융거래 정보를 요구하거나 안전조치 등을 명목으로 자금의 이체 등을 요구하는 경우는 100% 보이스피싱이므로 이러한 전화를 받는 경우 전화를 끊고 해당 기관의 대표전화로 전화하여 사실여부를 반드시 확인하시기 바랍니다.
② 검사의 약식명령 청구
검사는 폭행·상해사건이 벌금, 과료 또는 몰수의 형벌에 해당하여 공판절차 없이 약식명령으로 하는 것이 적당하다고 인정되는 경우에는 지방법원에 공소 제기와 동시에 서면으로 약식명령을 청구합니다(「형사소송법」 제448조제1항, 제449조 및 제450조).
약식명령을 청구 받은 지방법원은 해당 사건이 약식명령으로 할 수 없거나 약식명령으로 하는 것이 적당하지 않다고 인정되는 경우에는 공판절차에 따라 심판합니다(「형사소송법」 제450조).
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.
④ 약식명령의 고지
약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지됩니다(「형사소송법」 제452조).
③ 약식명령의 방식
지방법원은 검사의 약식명령의 청구를 인정하는 경우 청구가 있은 날부터 14일 이내에 약식명령을 하며(「형사소송규칙」제171조), 약식명령에는 범죄사실, 

In [ ]:
suspect_message = "만약 당신이 정말로 구속수사는 면하고 싶다 그러시면 본인 담당검사 있죠? 메모하세요. 신원보증 서류, 개인의견서 이 두가지 서류를 금융감독원으로 보내달라고 하시고 제일 중요한 거는 담당 검사가 당신을 신원보증 서주는 게 제일 중요합니다. 만약 담당 검사가 본인을 신원보증 서주겠다고 그러면 제가 약식명령으로 생각을 다시 한번 해볼께요. 근데 아마 담당 검사가 그럴 일은 없겠죠? 혐의자 신분인 본 인한테 뭘 믿고 보증을 서주겠어요? 일단 담당검사한테 울고불고 부탁을 해서라도 신원보증 받는 게 제 일 중요합니다."

relevant_docs = retriever.invoke(suspect_message)

print("paraphrase-multilingual-MiniLM-L12-v2 임베딩 모델")
print(f"총 {len(relevant_docs)}개의 문서가 검색되었습니다.")
for i, doc in enumerate(relevant_docs):
    print(doc.page_content)

paraphrase-multilingual-MiniLM-L12-v2 임베딩 모델
총 10개의 문서가 검색되었습니다.
② 검사의 약식명령 청구
검사는 폭행·상해사건이 벌금, 과료 또는 몰수의 형벌에 해당하여 공판절차 없이 약식명령으로 하는 것이 적당하다고 인정되는 경우에는 지방법원에 공소 제기와 동시에 서면으로 약식명령을 청구합니다(「형사소송법」 제448조제1항, 제449조 및 제450조).
약식명령을 청구 받은 지방법원은 해당 사건이 약식명령으로 할 수 없거나 약식명령으로 하는 것이 적당하지 않다고 인정되는 경우에는 공판절차에 따라 심판합니다(「형사소송법」 제450조).
① 약식명령의 개념
약식명령이란, 공판절차 없이 약식절차에 따라 벌금, 과료 또는 몰수 등 재산형을 과하는 지방법원의 재판을 말하며, 약식명령 시에는 추징, 그 밖의 부수의 처분도 할 수 있습니다(법령용어사례집, 법제처·한국법제연구원, 2003).
④ 약식명령의 고지
약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지됩니다(「형사소송법」 제452조).
① 전화로 정부기관이라며 자금이체를 요구하면 일단 보이스피싱 의심
검찰∙경찰∙금감원 등 정부기관은 어떠한 경우에도 전화로 자금의 이체 또는 개인의 금융거래정보를 요구하지 않습니다. 정부기관을  사칭, 범죄에 연루되었다며 금융거래 정보를 요구하거나 안전조치 등을 명목으로 자금의 이체 등을 요구하는 경우는 100% 보이스피싱이므로 이러한 전화를 받는 경우 전화를 끊고 해당 기관의 대표전화로 전화하여 사실여부를 반드시 확인하시기 바랍니다.
⑤ 효력
약식명령은 1. 정식재판의 청구기간이 경과한 때, 2. 정식재판 청구의 취하의 결정이 확정된 때, 3. 정식재판 청구기각의 결정이 확정된 때에는 확정 판결과 동일한 효력을 갖습니다(「형사소송법」 제457조).
③ 체포ㆍ구속ㆍ압수 또는 수색을 할 때에는 적법한 절차에 따라 검사의 신청에 의하여 법관이 발부한 영장을 제시하여야 한다. 다만, 현행범인인 경우와 장기 3년 이상의 형에 해당하는 죄를 범하고 도피

### ⓔ Reranking 모델
- BAAI/bge-reranker-v2-m3 (고성능 리랭킹 다국어 모델)

In [ ]:
#!pip install --upgrade langchain

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document

reranker_model_name = "BAAI/bge-reranker-v2-m3"
# Cross-encoder 모델 초기화
cross_encoder_model = HuggingFaceCrossEncoder(
    model_name=reranker_model_name,
    model_kwargs={"device": "cuda"}
)

# reranker 인스턴스 생성
compressor = CrossEncoderReranker(
    model=cross_encoder_model,
    top_n=2
)

# ContextualCompressionRetriever로 래핑
compression_retriever = ContextualCompressionRetriever(
    base_retriever=retriever,
    base_compressor=compressor
)

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

- 사례1: 발설하면안된다(누구든지 고발가능). 너 발설하면 가중처벌이다(협박성-> 증거채택X)

In [ ]:
reranked_docs = compression_retriever.invoke(suspect_message)

In [ ]:
reranked_docs

[Document(id='943c9c24-c2d4-454e-b9ec-2e23092889a7', metadata={'source': '헌법제12조', 'section_id': 'CONST12-⑦', 'title': '피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.'}, page_content='⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.'),
 Document(id='442d79a5-e029-4e12-8df3-77adeb253d44', metadata={'source': '형사소송법', 'section_id': 'CPL-본문', 'title': '형사소송법'}, page_content='제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.')]

- 사례2: 너 구속수사 당하기 싫으면 서류보내라.그래야 약식수사로 전환하겠다.

In [ ]:
reranked_docs = compression_retriever.invoke(suspect_message)
reranked_docs

[Document(id='7e4600a7-6335-41c6-a83a-6f8575399970', metadata={'source': '약식명령', 'section_id': 'YAK-④', 'title': '약식명령의 고지'}, page_content='④ 약식명령의 고지\n약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지됩니다(「형사소송법」 제452조).'),
 Document(id='2eb8312f-9df5-459f-bc5d-9325f50293bf', metadata={'source': '약식명령', 'section_id': 'YAK-③', 'title': '약식명령의 방식'}, page_content='③ 약식명령의 방식\n지방법원은 검사의 약식명령의 청구를 인정하는 경우 청구가 있은 날부터 14일 이내에 약식명령을 하며(「형사소송규칙」제171조), 약식명령에는 범죄사실, 적용법령, 주형(主刑), 부수 처분과 약식명령의 고지를 받은 날부터 7일 이내에 정식재판의 청구를 할 수 있다는 사실이 명시됩니다(「형사소송법」 제451조).')]

# 3. LLM 연결 (LangChain)

In [13]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ============================
# 0. 필수 설정 및 데이터 준비
# ============================
MODEL_ID = "kakaocorp/kanana-1.5-8b-instruct-2505"

USER_TEMPLATE = (
    "당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, "
    "공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.\n\n"
    "다음 형식을 지키세요:\n"
    "1) 경고 메시지\n"
    "   - 예시: \"이 내용은 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다!\"\n"
    "2) 참고 문서별 근거 설명 (무조건 세 가지 문서에 대해 각각 분리하여 설명)\n"
    "   - 첫 번째 문서 근거 설명\n"
    "   - 두 번째 문서 근거 설명\n"
    "   - 세 번째 문서 근거 설명\n\n"
    "참고 문서:\n{context}\n\n"
    "의심 문장:\n{question}\n"
)

suspect = "지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다.당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?"

# ============================
# LLM 모델과 토크나이저 로드
# ============================
tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.eval()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    max_new_tokens=512,
)
llm = HuggingFacePipeline(pipeline=pipe)

# ============================
# LLM & RAG 연결
# ============================
# 프롬프트 템플릿 객체 생성
PROMPT = PromptTemplate(
    template=USER_TEMPLATE,
    input_variables=["context", "question"]
)

# RetrievalQA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# RAG 체인 실행
# 'question' 키를 사용하여 의심 문장을 전달
result = qa_chain.invoke({"query": suspect})

print("\n" + "="*50)
print("사례:", suspect)
print("-" * 50)
print("생성된 답변:\n", result["result"])
print("="*50)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0



사례: 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다.당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?
--------------------------------------------------
생성된 답변:
 당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, 공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.

다음 형식을 지키세요:
1) 경고 메시지
   - 예시: "이 내용은 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다!"
2) 참고 문서별 근거 설명 (무조건 세 가지 문서에 대해 각각 분리하여 설명)
   - 첫 번째 문서 근거 설명
   - 두 번째 문서 근거 설명
   - 세 번째 문서 근거 설명

참고 문서:
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.

⑤ 누구든지 체포 또는 구속의 이유와 변호인의 조력을 받을 권리가 있음을 고지받지 아니하고는 체포 또는 구속을 당하지 아니한다. 체포 또는 구속을 당한 자의 가족등 법률이 정하는 자에게는 그 이유와 일시ㆍ장소가 지체없이 통지되어야 한다.

제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.

의심 문장:
지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다.당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?
---
1) 경고 메시지
이 내용은 명백한 보이스피싱 수법인 '협박과 위협을 통한 개인정보 유출 유도'가 의심됩니다!

2) 참고

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document

# ============================
# 0. 필수 설정 및 데이터 준비
# ============================
MODEL_ID = "kakaocorp/kanana-1.5-8b-instruct-2505"

USER_TEMPLATE = (
    "당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, "
    "공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.\n\n"
    "다음 형식을 반드시 지키세요:\n"
    "1) 경고 메시지\n"
    "   - 예시: \"이 내용은 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다!\"\n"
    "2) 참고 문서별 근거 설명 (무조건 두 가지 문서에 대해 각각 분리하여 설명)\n"
    "   - 첫 번째 문서 근거 설명\n"
    "   - 두 번째 문서 근거 설명\n\n"
    "참고 문서:\n{context}\n\n"
    "의심 문장:\n{question}\n"
)


suspect = "본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?"

# ============================
# 3. LLM 모델 (Kanana) 로드
# ============================
tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto",
)
model.eval()
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    max_new_tokens=512,
)
llm = HuggingFacePipeline(pipeline=pipe)

# ============================
# 4. LLM & RAG 체인 연결 및 실행
# ============================
PROMPT = PromptTemplate(
    template=USER_TEMPLATE,
    input_variables=["context", "question"]
)

# RetrievalQA 체인 구성: LLM과 압축 검색기(리랭커 포함) 연결
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# RAG 체인 실행
result = qa_chain.invoke({"query": suspect})

# 결과 출력
print("생성된 답변:\n", result["result"])
print("="*50)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


생성된 답변:
 당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, 공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.

다음 형식을 반드시 지키세요:
1) 경고 메시지
   - 예시: "이 내용은 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다!"
2) 참고 문서별 근거 설명 (무조건 두 가지 문서에 대해 각각 분리하여 설명)
   - 첫 번째 문서 근거 설명
   - 두 번째 문서 근거 설명

참고 문서:
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.

제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.

의심 문장:
본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?
---
이 문장은 명백한 보이스피싱 수법인 '공포심 유발 및 불이익 협박'이 의심됩니다!
  
① 형법 제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다. 이 조항은 누구나 범죄 사실을 알게 된 경우 고발할 권리가 있음을 명시하고 있습니다. 따라서 피해자가 법적 조치를 취할 수 없다는 주장은 사실과 다릅니다.

② 형법 제234조의2(공무원 등의 범죄)에 따르면, 공무원 등이 직무와 관련하여 범죄를 저지른 경우 가중처벌을 받을 수 있다고 규정하고 있습니다. 또한, 형법 제234조에 따라 범죄 사실을 고발하는 행위는 합법적인 권리입니다. 보이스피싱 범죄자들은 피해자를 혼란스럽게 하기 위

# 4. QWEN 모델과 연결

- 사례1: Rag 연결 전 대답

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

system_prompt = "너는 한국어 보이스피싱 탐지기야. 한국어만 사용."
user_prompt = f"""
다음 문장을 보고 실제 공공기관의 정상 절차와 비교해 이상한 점을 설명하고,
어떤 보이스피싱 수법인지 1-2문장으로 규정해라.
출력 형식은 아래와 같이 해라:
1. 이상한 점: ...
2. 보이스피싱 수법: ...
보이스피싱 문장: {suspect_message}
"""
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
response

'1. 이상한 점: 이 문장에서는 피해자가 아닌 혐의자로서 행동해야 한다는 내용이 언급되어 있습니다. 실제 공공기관의 정상 절차에서는 혐의자와 피해자의 역할이 명확하게 구분되어 있습니다.\n2. 보이스피싱 수법: 이 문장은 혐의자로서 가중처벌을 받을 것이라는 협박과 함께 가족이나 지인에게 정보를 제공하면 모두가 처벌받을 것이라는 심리적 압박을 이용한 보이스피싱 수법입니다.'

- 사례2: Rag 연결 전 대답

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

system_prompt = "너는 한국어 보이스피싱 탐지기야. 한국어만 사용."
user_prompt = f"""
다음 문장을 보고 실제 공공기관의 정상 절차와 비교해 이상한 점을 설명하고,
어떤 보이스피싱 수법인지 1-2문장으로 규정해라.
출력 형식은 아래와 같이 해라:
1. 이상한 점: ...
2. 보이스피싱 수법: ...
보이스피싱 문장: {suspect_message}
"""
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
response

'1. 이상한 점: 문장에서 금융감독원이라는 공공기관에 직접 서류를 보내라고 요구하고 있으며, 실제로 금융감독원은 이런 직접적인 서류 전달 절차는 존재하지 않습니다.\n2. 보이스피싱 수법: 이 문장은 혐의자가 있는 사람에게 금융감독원으로 서류를 보내라는 명령을 내리며, 이를 통해 보이스피싱 범죄자의 신원보증 서류 요청을 위장하여 돈을 요구하는 수법입니다.'

------------------------
- 사례1: Rag 연결 후 대답

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document

# ============================
# 1. Qwen 모델에 재순위화 결과 연결
# ============================

# 재순위화된 문서의 내용을 하나의 문자열로 결합 -> 이 부분이 Qwen 모델에게 넘겨줄 "참고 문서"가 됩니다.
context_from_docs = "\n\n".join([doc.page_content for doc in reranked_docs])

system_prompt = (
    "당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, "
    "공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.\n\n"
    "설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. "
    "각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. "
    "지시된 출력 JSON 이외의 말은 하지 않는다."
)

user_prompt_template = """
다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: \"이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! \")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
{context}
---
보이스피싱 문장:
{suspect_message}
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
"""

# 프롬프트 템플릿에 변수를 채워 넣어 최종 프롬프트 문자열 생성
user_prompt = user_prompt_template.format(
    context=context_from_docs,
    suspect_message=suspect_message
)

# ============================
# 2. Qwen 모델 호출 및 결과 출력
# ============================
model_name = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

- 보이스피싱 수법: 이 내용은 명백한 보이스피싱 수법 '[기망]'을 활용한 것 같습니다. 피해자가 혐의자 신분임을 암시하며, 그에 따른 제재를 언급해 두려움을 조성하고 있습니다.
- 각 참고 문서별 근거로 왜 잘못됐는지: 
[⑦] 고문이나 폭행, 협박, 구속의 부당한 장기화 등 방법에 의하여 진술이 자의로 이루어진 것은 인정되지 않습니다. 따라서 이 문장에서의 '고문ㆍ폭행ㆍ협박'은 명백히 불법이며, 피해자의 진술이 무효가 됩니다. 또한, '기망'이라는 용어는 피해자를 두려움 속으로 끌어들여 진실을 숨기는 것을 암시합니다. 이는 법률적으로 인정받을 수 없는 수법입니다.
[제234조(고발)] 고발자는 자신이 범죄를 알았을 때 고발할 수 있으며, 이는 진실을 밝히는 방안입니다. 그러나 이 문장은 고발자의 입장에서 보이지 않고, 오히려 피해자에게 고발을 하면 처벌을 받을 수 있다는 위협을 주고 있습니다. 이는 사실상 위협과 협박으로 해석되며, 이러한 방법은 법적으로 문제가 됩니다.


In [ ]:
print(text) # 프롬프트 확인

<|im_start|>system
당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, 공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.

설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. 각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. 지시된 출력 JSON 이외의 말은 하지 않는다.<|im_end|>
<|im_start|>user

다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: "이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! ")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.

제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.
---
보이스피싱 문장:
본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 안된다는 겁니다. 읽은 것을 증거자료로 보내왔어요. 당신이 엠바고 사건 관련해서 가족 지인한테 발설했을 경우에는 어떻게 될거 같아요. 가중처벌은 당연한거고 당신이 발설한 그 사람들도 같이 처벌 받아요. 이해하셨죠?
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
<|im_end|>
<|im_start|>assistant



- 사례2: Rag 연결 후 대답

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document

# ============================
# 1. Qwen 모델에 재순위화 결과 연결
# ============================

# 재순위화된 문서의 내용을 하나의 문자열로 결합 -> 이 부분이 Qwen 모델에게 넘겨줄 "참고 문서"가 됩니다.
context_from_docs = "\n\n".join([doc.page_content for doc in reranked_docs])

system_prompt = (
    "당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, "
    "공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.\n\n"
    "설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. "
    "각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. "
    "지시된 출력 JSON 이외의 말은 하지 않는다."
    "한국어로만 답변한다"
)

user_prompt_template = """
다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: \"이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! \")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
{context}
---
보이스피싱 문장:
{suspect_message}
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
"""

# 프롬프트 템플릿에 변수를 채워 넣어 최종 프롬프트 문자열 생성
user_prompt = user_prompt_template.format(
    context=context_from_docs,
    suspect_message=suspect_message
)

# ============================
# 2. Qwen 모델 호출 및 결과 출력
# ============================
model_name = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

- 보이스피싱 수법: 명확한 약식명령 절차와 관련된 정보를 이용해 불법적인 방법으로 구속수사를 면하려는 수법.
- 각 참고 문서별 근거로 왜 잘못됐는지: 
[④] 약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지되는 것이므로, 이를 이용해 서류를 직접 제출하거나 서명을 강요하는 것은 불법입니다. [③]에서는 지방법원이 검사의 약식명령 청구를 인정하면, 해당 날로부터 14일 이내에 약식명령을 내리지만, 이를 이용해 서류를 보내거나 직접 제출하는 것은 불법적입니다. 또한, 약식명령에는 정식재판 청구가 가능하다는 내용이 명시되어 있지만, 이를 이용해 불법적인 서류 요청을 하는 것은 불법적입니다. 이 모든 점에서, 위의 보이스피싱 문장은 명백한 보이스피싱 수법이며, 법적으로 위법한 행위입니다.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.documents import Document

# ============================
# 1. Qwen 모델에 재순위화 결과 연결
# ============================

# 재순위화된 문서의 내용을 하나의 문자열로 결합 -> 이 부분이 Qwen 모델에게 넘겨줄 "참고 문서"가 됩니다.
context_from_docs = "\n\n".join([doc.page_content for doc in reranked_docs])

system_prompt = (
    "당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, "
    "공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.\n\n"
    "설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. "
    "각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. "
    "지시된 출력 JSON 이외의 말은 하지 않는다."
)

user_prompt_template = """
다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: \"이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! \")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
{context}
---
보이스피싱 문장:
{suspect_message}
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
"""

# 프롬프트 템플릿에 변수를 채워 넣어 최종 프롬프트 문자열 생성
user_prompt = user_prompt_template.format(
    context=context_from_docs,
    suspect_message=suspect_message
)

# ============================
# 2. Qwen 모델 호출 및 결과 출력
# ============================
model_name = "Qwen/Qwen2.5-3B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

- 보이스피싱 수법: 약식명령 요구 및 신원보증 서류 요청
- 각 참고 문서별 근거로 왜 잘못됐는지: 
  - 참고 문서 ③ 약식명령의 방식에 따르면, 지방법원은 검사의 약식명령 청구를 인정하는 경우에만 약식명령을 내리게 되며, 이를 위해서는 검사가 실제로 약식명령을 청구해야 합니다. [③ 약식명령의 방식]
  - 또한, 약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지되지만, 이는 법정에서의 절차로, 보이스피싱 상황에서는 이러한 절차를 거치지 않습니다. [④ 약식명령의 고지]
  - 따라서 이 문장에서 제시하는 약식명령 요구와 신원보증 서류 요청은 실제 법적 절차와 상관없는 무단 요구로, 명백한 보이스피싱 수법입니다.


In [ ]:
print(text) # 프롬프트 확인

<|im_start|>system
당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, 공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.

설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. 각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. 지시된 출력 JSON 이외의 말은 하지 않는다.<|im_end|>
<|im_start|>user

다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: "이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! ")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
④ 약식명령의 고지
약식명령은 검사와 피고인에게 재판서를 송달함으로써 고지됩니다(「형사소송법」 제452조).

③ 약식명령의 방식
지방법원은 검사의 약식명령의 청구를 인정하는 경우 청구가 있은 날부터 14일 이내에 약식명령을 하며(「형사소송규칙」제171조), 약식명령에는 범죄사실, 적용법령, 주형(主刑), 부수 처분과 약식명령의 고지를 받은 날부터 7일 이내에 정식재판의 청구를 할 수 있다는 사실이 명시됩니다(「형사소송법」 제451조).
---
보이스피싱 문장:
만약 당신이 정말로 구속수사는 면하고 싶다 그러시면 본인 담당검사 있죠? 메모하세요. 신원보증 서류, 개인의견서 이 두가지 서류를 금융감독원으로 보내달라고 하시고 제일 중요한 거는 담당 검사가 당신을 신원보증 서주는 게 제일 중요합니다. 만약 담당 검사가 본인을 신원보증 서주겠다고 그러면 제가 약식명령으로 생각을 다시 한번 해볼께요. 근데 아마 담당 검사가 그럴 일은 없겠죠? 혐의자 신분인 본 인한테 뭘 믿고 보증을 서주겠어요? 일단 담당검사한테 울고불고 부탁을 해서라도 신원보증 받는 게 제 일 중요합니다.
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
<|im_end|>
<|im_st

# 5. 파인튜닝한 QWEN 모델과 연결

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# ====== 사용자 수정 영역 ======
OUTPUT_DIR = "/content/drive/MyDrive/KDH/LLM/lora_keyword_only"
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"
# ============================

# 1. 원본 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

# 2. 파인튜닝된 LoRA 어댑터 로드 및 결합
lora_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
lora_model.eval() # 추론 모드로 전환

print("✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.


- 사례 1

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(lora_model.device)

generated_ids = lora_model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

- 보이스피싱 수법: 이 내용은 명백한 보이스피싱 수법 "불법적인 혐의자 신분 확인 및 정보 제공 요구"를 이용한 것입니다.
- 각 참고 문서별 근거로 왜 잘못됐는지: 
[⑦] 피고인의 자백이 불법적인 방법에 의해 이루어진 것이 아니라는 점을 인정받으면, 이를 유죄의 증거로 삼거나 처벌할 수 없습니다. 이 문장에서 제안하는 내용은 불법적인 방법으로 혐의자 신분 확인을 요구하고 있으므로, 이는 불법적이고 위법한 수법입니다. 또한, [제234조(고발)]에서 명시된 것처럼 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있지만, 이 문장은 고발을 강요하거나, 이를 통해 혐의자 신분 확인을 요구하는 것은 위법하다고 볼 수 있습니다. 이로 인해 피해자는 불법적인 방법으로 정보를 제공해야 하는 강요를 받게 됩니다.


- 사례 2

In [ ]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(lora_model.device)

generated_ids = lora_model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

- 보이스피싱 수법: 명백한 보이스피싱 수법으로, 약식명령 과정에서 검사가 피고인을 신원보증 서류를 요구하는 것은 불가능하다는 이유로 분석됩니다.
- 각 참고 문서별 근거로 왜 잘못됐는지: 
[④] 약식명령의 고지는 검사와 피고인에게 재판서를 송달함으로써 이루어집니다. 여기서 '재판서'는 재판 결과물 또는 재판 진행 상황을 알려주는 문서를 의미하지만, 약식명령은 피고인의 혐의에 대한 단순 결정이며, 이를 통해 피고인에게 직접 서류를 보여주거나 신원보증 서류를 요구하는 것은 불가능합니다. [③] 약식명령의 방식에서는 검사가 약식명령을 청구한 날로부터 14일 이내에 약식명령을 내리지만, 약식명령 자체는 피고인에게 직접 서류를 보여주지 않고, 피고인에게 약식명령의 존재를 알리는 것으로만 처리됩니다. 따라서 피고인에게 신원보증 서류를 요구하는 것은 불법적이고, 또한 위임을 받아서 서류를 작성하거나 서명하는 것도 불가능합니다.


 # 6. 프롬프트 테스트 (QWEN 2.5 3B INSTRUCT + analysis 파인튜닝 모델)

```
[FEW SHOT 방식]
긴 버전
<context>
[형사소송법 제234조 1항] 피해자는 범죄 사실을 수사기관에 신고할 수 있다
</context>
<utterance>피해자가 가족에게 발설하면 가중처벌되고 지인도 같이 처벌받아요.</utterance>
<output>피해자가 가족에게 알렸다고 가중처벌은 존재하지 않습니다 (형사소송법 제234조)</output>
<output>제3자가 같이 처벌받는다는 주장은 법적 근거가 없습니다 (형사소송법 제234조)</output>

<context>
[전기통신금융사기 피해금 환급 특별법 제5조] 피해금 환급 절차 규정
</context>
<utterance>국가전산자금으로 불법 대출을 완납 처리해줍니다.</utterance>
<output>국가전산자금 제도는 없으며 피해금 환급 절차만 존재합니다 (전기통신금융사기 피해금 환급 특별법 제5조)</output>
<output>불법 대출 피해금은 금융기관을 통한 환급 절차로만 처리됩니다 (전기통신금융사기 피해금 환급 특별법 제5조)</output>

<context>
[형법 제123조 직권남용] 공무원은 직권을 남용하여 의무없는 일을 강요할 수 없음
</context>
<utterance>본인 신분증을 얼굴 옆에 들고 영상 촬영해 카톡으로 보내세요.</utterance>
<output>공무원은 신분증 영상 제출을 강요할 권한이 없습니다 (형법 제123조)</output>
<output>신분증 영상 촬영 요구는 직권남용에 해당하므로 응하면 안 됩니다 (형법 제123조)</output>
```


### ✅ FEW SHOT 프롬프트 방식

In [ ]:
reranked_docs

[Document(id='943c9c24-c2d4-454e-b9ec-2e23092889a7', metadata={'source': '헌법제12조', 'section_id': 'CONST12-⑦', 'title': '피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.'}, page_content='⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.'),
 Document(id='442d79a5-e029-4e12-8df3-77adeb253d44', metadata={'source': '형사소송법', 'section_id': 'CPL-본문', 'title': '형사소송법'}, page_content='제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.')]

In [ ]:
 # ============================
# 1. Qwen 모델에 재순위화 결과 연결
# ============================

# 재순위화된 문서의 내용을 하나의 문자열로 결합 -> 이 부분이 Qwen 모델에게 넘겨줄 "참고 문서"가 됩니다.
context_from_docs = "\n\n".join([doc.page_content for doc in reranked_docs])

system_prompt = (
    "당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, "
    "공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.\n\n"
    "설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. "
    "각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. "
    "지시된 출력 JSON 이외의 말은 하지 않는다."
)

user_prompt_template = """
다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: \"이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! \")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
{context}
---
보이스피싱 문장:
{suspect_message}
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
"""

user_prompt = user_prompt_template.format(
    context=context_from_docs,
    suspect_message=suspect_message
)

messages = [
    {"role": "system", "content": system_prompt},

    # FEW-SHOT 1
    {"role": "user", "content":
     "참고 문서:\n[형사소송법 제234조 1항] 피해자는 범죄 사실을 수사기관에 신고할 수 있다\n"
     "보이스피싱 문장:\n피해자가 가족에게 발설하면 가중처벌되고 지인도 같이 처벌받아요."},
    {"role": "assistant", "content":
     "피해자가 가족에게 알렸다고 가중처벌은 존재하지 않습니다 (형사소송법 제234조)\n"
     "제3자가 같이 처벌받는다는 주장은 법적 근거가 없습니다 (형사소송법 제234조)"},

    # FEW-SHOT 2
    {"role": "user", "content":
     "참고 문서:\n[전기통신금융사기 피해금 환급 특별법 제5조] 피해금 환급 절차 규정\n"
     "보이스피싱 문장:\n국가전산자금으로 불법 대출을 완납 처리해줍니다."},
    {"role": "assistant", "content":
     "국가전산자금 제도는 없으며 피해금 환급 절차만 존재합니다 (전기통신금융사기 피해금 환급 특별법 제5조)\n"
     "불법 대출 피해금은 금융기관을 통한 환급 절차로만 처리됩니다 (전기통신금융사기 피해금 환급 특별법 제5조)"},

    # FEW-SHOT 3
    {"role": "user", "content":
     "참고 문서:\n[형법 제123조 직권남용] 공무원은 직권을 남용하여 의무없는 일을 강요할 수 없음\n"
     "보이스피싱 문장:\n본인 신분증을 얼굴 옆에 들고 영상 촬영해 카톡으로 보내세요."},
    {"role": "assistant", "content":
     "공무원은 신분증 영상 제출을 강요할 권한이 없습니다 (형법 제123조)\n"
     "신분증 영상 촬영 요구는 직권남용에 해당하므로 응하면 안 됩니다 (형법 제123조)"},

    {"role": "user", "content": user_prompt}
]

OUTPUT_DIR = "/content/drive/MyDrive/KDH/LLM/lora_keyword_only"
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

# 1. 원본 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

# 2. 파인튜닝된 LoRA 어댑터 로드 및 결합
lora_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
lora_model.eval() # 추론 모드로 전환
print("✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.")


text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(lora_model.device)

generated_ids = lora_model.generate(
    **model_inputs,
    max_new_tokens=512
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.
- 보이스피싱 수법: 이 내용은 "사람을 혐의자로 만드는 기망"한 이유로, 명백한 보이스피싱 수법인 "사기와 혐의자 만들기"가 의심됩니다!
- 각 참고 문서별 근거로 왜 잘못됐는지: 
인용: [형법 제123조 직권남용] 공무원은 명시적으로 명령하지 않는 한 명의 행위를 강요할 수 없습니다. 따라서 피고인은 혐의자를 만들기 위한 명령이나 강요를 할 수 없습니다 (형법 제123조). 
인용: [전기통신금융사기 피해금 환급 특별법 제5조] 피해자에게 피해금 환급 절차는 있지만, 명시적으로 '모든 정보를 유출하지 않도록' 요구하거나 처벌하는 것은 법적 근거가 없습니다 (전기통신금융사기 피해금 환급 특별법 제5조).
인용: [형사소송법 제234조 1항] 피해자는 범죄 사실을 고발할 수 있으나, 고발이 유죄의 증거로 취급되거나 처벌 요인이 될 수 없습니다 (형사소송법 제234조).


In [ ]:
print(text)

<|im_start|>system
당신은 보이스피싱 피해를 막기 위해 의심스러운 대화 내용을 분석하고, 공식 문서를 바탕으로 위험성을 해설하는 전문가입니다.

설명은 일반인이 이해할 수 있게 쉽게 쓰고, 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙인다. 각 주장에는 참고 문서 번호 [n]로 근거를 표시한다. 지시된 출력 JSON 이외의 말은 하지 않는다.<|im_end|>
<|im_start|>user
참고 문서:
[형사소송법 제234조 1항] 피해자는 범죄 사실을 수사기관에 신고할 수 있다
보이스피싱 문장:
피해자가 가족에게 발설하면 가중처벌되고 지인도 같이 처벌받아요.<|im_end|>
<|im_start|>assistant
피해자가 가족에게 알렸다고 가중처벌은 존재하지 않습니다 (형사소송법 제234조)
제3자가 같이 처벌받는다는 주장은 법적 근거가 없습니다 (형사소송법 제234조)<|im_end|>
<|im_start|>user
참고 문서:
[전기통신금융사기 피해금 환급 특별법 제5조] 피해금 환급 절차 규정
보이스피싱 문장:
국가전산자금으로 불법 대출을 완납 처리해줍니다.<|im_end|>
<|im_start|>assistant
국가전산자금 제도는 없으며 피해금 환급 절차만 존재합니다 (전기통신금융사기 피해금 환급 특별법 제5조)
불법 대출 피해금은 금융기관을 통한 환급 절차로만 처리됩니다 (전기통신금융사기 피해금 환급 특별법 제5조)<|im_end|>
<|im_start|>user
참고 문서:
[형법 제123조 직권남용] 공무원은 직권을 남용하여 의무없는 일을 강요할 수 없음
보이스피싱 문장:
본인 신분증을 얼굴 옆에 들고 영상 촬영해 카톡으로 보내세요.<|im_end|>
<|im_start|>assistant
공무원은 신분증 영상 제출을 강요할 권한이 없습니다 (형법 제123조)
신분증 영상 촬영 요구는 직권남용에 해당하므로 응하면 안 됩니다 (형법 제123조)<|im_end|>
<|im_start|>user

다음 형식을 지키세요:
1) 사

### ✅ COT 프롬프트 방식

```
[COT 방식]

ROLE_BLOCK = """
# 역할너는 보이스피싱 탐지 AI 전문가야. 너의 임무는
주어진 통화 내용의 맥락과 표현을 분석하여 보이스피싱 사기 위험도를 정확하게 판단하는 것이다."""

GUIDE_BLOCK = """# 지시아래 "통화 내용"을 "분석 단계"에 따라 체계적으로 점검하고,
결과를 "출력 형식"에 맞춰 간결하게 작성해라.
사고 과정의 세부 연쇄 사고는 내부에서만 수행하고 출력하지 마라."""

ANALYSIS_STEPS = """# 분석 단계1.  
**화자 신분 확인:** 공공기관/지인 사칭 여부.
**긴급성·위협:** 다급함/처벌·구속 등 압박 언어 사용 여부.  
**요구 사항:** 개인정보/금융정보 요구 또는 이체 요청 여부.  
**비상식적 요구:** 원격앱 설치, 의심 링크 클릭 등.
 **종합 결론:** 보이스피싱 가능성 판단"""


OUTPUT_FORMAT = """# 출력 형식* **분석 과정:**    
* 1. 화자 신분: [분석]    
* 2. 긴급성/위협: [분석]   
 * 3. 요구 사항: [분석]   
  * 4. 비상식적 행위 유도: [분석]*
  **요약:** [결론과 핵심 근거를 2-3문장으로 요약]"""



TASK_HEADER = """다음 통화 내용을 위 기준으로 평가하라."""

결과
* **분석 과정:**
    * 1. 화자 신분: [분석] - 보이스피싱 테러리스트의 사칭으로 추정됨
    * 2. 긴급성/위협: [분석] - 다급한 tone과 압박적인 언어 사용
    * 3. 요구 사항: [분석] - 금융 정보 요구 및 이체 요청 가능성이 있음
    * 4. 비상식적 행위 유도: [분석] - 원격 앱 설치나 의심 링크 클릭을 유도
* **요약:** 보이스피싱 가능성이 높음. 다급한 tone과 금융 정보 요구, 압박 언어, 비상식적 요구사항이 모두 확인됨.
```

In [ ]:
# ============================
# 1) COT 방식 프롬프트 블록
# ============================
ROLE_BLOCK = """
# 역할
너는 보이스피싱 탐지 AI 전문가야. 너의 임무는
주어진 통화 내용의 맥락과 표현을 분석하여 보이스피싱 사기 위험도를 정확하게 판단하는 것이다.
""".strip()

GUIDE_BLOCK = """
# 지시
아래 "통화 내용"을 "분석 단계"에 따라 체계적으로 점검하고,
결과를 "출력 형식"에 맞춰 간결하게 작성해라.
사고 과정의 세부 연쇄 사고는 내부에서만 수행하고 출력하지 마라.
""".strip()

ANALYSIS_STEPS = """
# 분석 단계
1. 화자 신분 확인: 공공기관/지인 사칭 여부
2. 긴급성·위협: 다급함/처벌·구속 등 압박 언어 사용 여부
3. 요구 사항: 개인정보/금융정보 요구 또는 이체 요청 여부
4. 비상식적 요구: 원격앱 설치, 의심 링크 클릭 등
5. 종합 결론: 보이스피싱 가능성 판단
""".strip()

OUTPUT_FORMAT = """
# 출력 형식
* **분석 과정:**
    * 1. 화자 신분: [분석] [n]
    * 2. 긴급성/위협: [분석] [n]
    * 3. 요구 사항: [분석] [n]
    * 4. 비상식적 행위 유도: [분석] [n]
* **요약:** [결론과 핵심 근거를 2-3문장으로 요약] [n]
""".strip()

TASK_HEADER = "다음 통화 내용을 위 기준으로 평가하라."

# 시스템 프롬프트: COT(내부), 출력은 고정 포맷만. [n] 인용 강제.
system_prompt = f"""
{ROLE_BLOCK}

{GUIDE_BLOCK}

{ANALYSIS_STEPS}

{OUTPUT_FORMAT}

# 규칙
- 반드시 위 '출력 형식'으로만 답하고, 다른 말은 하지 마라.
- 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙여라.
- 각 판단에는 참고 문서 번호 [n]를 인용하여 근거를 표시하라.
- 내부 사고(연쇄 사고 단계)는 출력하지 마라.
- 문서에 직접적 근거가 없으면 '근거 불충분'이라고 명시하라.
""".strip()

# ============================
# 2) 사용자 프롬프트(템플릿 -> 채움)
# ============================
# suspect_message: 통화 원문(문자/메신저 대화 포함)을 넣으세요.
user_prompt = f"""
{TASK_HEADER}

---
참고 문서:
{context_from_docs}
---
통화 내용:
{suspect_message}
""".strip()

# ============================
# 3) 모델 로드
# ============================
OUTPUT_DIR = "/content/drive/MyDrive/KDH/LLM/lora_keyword_only"
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

# LoRA 어댑터 결합
lora_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
lora_model.eval()
print("✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.")

# ============================
# 4) ChatML 생성 및 추론
# ============================
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

chatml_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True  # 마지막에 <|im_start|>assistant 자동 추가
)

model_inputs = tokenizer([chatml_text], return_tensors="pt").to(lora_model.device)

generated_ids = lora_model.generate(
    **model_inputs,
    max_new_tokens=512,
    eos_token_id=tokenizer.convert_tokens_to_ids("<|im_end|>"),
    pad_token_id=tokenizer.eos_token_id
)

# 프롬프트 토큰 제거
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.
* **분석 과정:**
    * 1. 화자 신분: [분석] 1
    * 2. 긴급성/위협: [분석] 0
    * 3. 요구 사항: [분석] 1
    * 4. 비상식적 행위 유도: [분석] 0
* **요약:** 보이스피싱 가능성이 매우 높습니다. 긴급성과 위협 요소 없이 혐의자 신분을 이용해 금융 정보 요구와 함께 가족 및 지인에 대한 가중 처벌 협박이 포함되어 있습니다. (1)


In [ ]:
print(chatml_text)

<|im_start|>system
# 역할
너는 보이스피싱 탐지 AI 전문가야. 너의 임무는
주어진 통화 내용의 맥락과 표현을 분석하여 보이스피싱 사기 위험도를 정확하게 판단하는 것이다.

# 지시
아래 "통화 내용"을 "분석 단계"에 따라 체계적으로 점검하고,
결과를 "출력 형식"에 맞춰 간결하게 작성해라.
사고 과정의 세부 연쇄 사고는 내부에서만 수행하고 출력하지 마라.

# 분석 단계
1. 화자 신분 확인: 공공기관/지인 사칭 여부
2. 긴급성·위협: 다급함/처벌·구속 등 압박 언어 사용 여부
3. 요구 사항: 개인정보/금융정보 요구 또는 이체 요청 여부
4. 비상식적 요구: 원격앱 설치, 의심 링크 클릭 등
5. 종합 결론: 보이스피싱 가능성 판단

# 출력 형식
* **분석 과정:**
    * 1. 화자 신분: [분석] [n]
    * 2. 긴급성/위협: [분석] [n]
    * 3. 요구 사항: [분석] [n]
    * 4. 비상식적 행위 유도: [분석] [n]
* **요약:** [결론과 핵심 근거를 2-3문장으로 요약] [n]

# 규칙
- 반드시 위 '출력 형식'으로만 답하고, 다른 말은 하지 마라.
- 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙여라.
- 각 판단에는 참고 문서 번호 [n]를 인용하여 근거를 표시하라.
- 내부 사고(연쇄 사고 단계)는 출력하지 마라.
- 문서에 직접적 근거가 없으면 '근거 불충분'이라고 명시하라.<|im_end|>
<|im_start|>user
다음 통화 내용을 위 기준으로 평가하라.

---
참고 문서:
[1] 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는

In [ ]:
# ============================
# 1) COT 방식 프롬프트(내부사고 비공개)
# ============================
ROLE_BLOCK = """
# 역할
너는 보이스피싱 탐지 AI 전문가야. 너의 임무는
주어진 보이스피싱범의 통화 내용의 맥락과 표현을 분석하여 보이스피싱 사기 위험도를 정확하게 판단하는 것이다.
한국어와 존댓말을 사용한다
""".strip()

GUIDE_BLOCK = """
# 지시
- 내부 사고(연쇄 사고)는 네 내부에서만 수행하고 출력하지 마라.
- 판단의 근거는 반드시 '참고 문서'에 번호[n]로 연결해서 표기해라.
- 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙여라.
- 사용자가 지정한 '출력 형식'을 그대로 지켜라. 불필요한 문장/형식 추가 금지.
""".strip()

ANALYSIS_STEPS = """
# 분석 단계(내부에서만 수행)
1) 화자 신분 사칭 여부
2) 긴급성/위협(압박) 언어
3) 요구 사항(개인정보/금융정보/이체)
4) 비상식적 요구(원격앱/의심 링크 등)
5) 종합 결론 도출
""".strip()

system_prompt = f"""
{ROLE_BLOCK}

{GUIDE_BLOCK}

{ANALYSIS_STEPS}
""".strip()

# ============================
# 2) 사용자 프롬프트 (네가 준 형식 유지)
# ============================
# suspect_message는 호출부에서 주입되어야 함
user_prompt_template = """
다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: "이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! ")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
{context}
---
보이스피싱 문장:
{suspect_message}
출력 형식은 아래와 같이 해라:
- 보이스피싱 수법: ...
- 각 참고 문서별 근거로 왜 잘못됐는지: ...
""".strip()

user_prompt = user_prompt_template.format(
    context=context_from_docs,
    suspect_message=suspect_message
)

# ============================
# 3) 모델 로드
# ============================
OUTPUT_DIR = "/content/drive/MyDrive/KDH/LLM/lora_keyword_only"
MODEL_NAME = "Qwen/Qwen2.5-3B-Instruct"

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype="auto",
    device_map="auto"
)

lora_model = PeftModel.from_pretrained(base_model, OUTPUT_DIR)
lora_model.eval()
print("✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.")

# ============================
# 4) ChatML 변환 및 추론
# ============================
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",  "content": user_prompt}
]

chatml_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

model_inputs = tokenizer([chatml_text], return_tensors="pt").to(lora_model.device)

generated_ids = lora_model.generate(
    **model_inputs,
    max_new_tokens=512,
    eos_token_id=tokenizer.convert_tokens_to_ids("<|im_end|>"),
    pad_token_id=tokenizer.eos_token_id
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 파인튜닝된 모델이 성공적으로 로드되었습니다.
- 보이스피싱 수법: "비상식적 요구" (참고문서 [4])
- 각 참고 문서별 근거로 왜 잘못됐는지: 
참고문서 [1]에 따르면, 피고인의 자백이 고문, 폭행, 협박, 구속의 부당한 장기화 또는 기망 등의 방법으로 자의로 진술된 것이 아니라고 인정되면 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다. 본 문장에서 혐의자 신분이기 때문에 엠바고 사건에 대한 정보를 누설하면 가중처벌이 될 것이라는 비상식적인 요구는 이러한 방법으로 인한 자의적인 진술을 의미한다. 또한, 참조문서 [2]의 제234조(고발)에 따르면, 누구든지 범죄가 있다고 사료하는 경우 고발할 수 있다. 이 문장에서 피해자로서 고발할 의향이 있는 상황이 아니라 오히려 피해자의 정보를 누설하는 것을 강요하는 것은 고발에 해당하지 않으며, 이는 고발에 해당하지 않는다고 판단된다.


In [ ]:
print(chatml_text)

<|im_start|>system
# 역할
너는 보이스피싱 탐지 AI 전문가야. 너의 임무는
주어진 통화/문장 내용의 맥락과 표현을 분석하여 보이스피싱 사기 위험도를 정확하게 판단하는 것이다.

# 지시
- 내부 사고(연쇄 사고)는 네 내부에서만 수행하고 출력하지 마라.
- 판단의 근거는 반드시 '참고 문서'에 번호[n]로 연결해서 표기해라.
- 법률/전문 용어가 나오면 괄호로 한 줄 풀이를 덧붙여라.
- 사용자가 지정한 '출력 형식'을 그대로 지켜라. 불필요한 문장/형식 추가 금지.

# 분석 단계(내부에서만 수행)
1) 화자 신분 사칭 여부
2) 긴급성/위협(압박) 언어
3) 요구 사항(개인정보/금융정보/이체)
4) 비상식적 요구(원격앱/의심 링크 등)
5) 종합 결론 도출<|im_end|>
<|im_start|>user
다음 형식을 지키세요:
1) 사용된 보이피싱 수법 분석(예: "이 내용은 ~한 이유로, 명백한 보이스피싱 수법인 '[주요 의심 수법]'이 의심됩니다! ")
2) 참고 문서를 근거로 왜 거짓인지 일반인 고려하여 쉽게 설명(인용: )
---
참고 문서:
[1] 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.
⑦ 피고인의 자백이 고문ㆍ폭행ㆍ협박ㆍ구속의 부당한 장기화 또는 기망 기타의 방법에 의하여 자의로 진술된 것이 아니라고 인정될 때 또는 정식재판에 있어서 피고인의 자백이 그에게 불리한 유일한 증거일 때에는 이를 유죄의 증거로 삼거나 이를 이유로 처벌할 수 없다.

[2] 형사소송법
제234조(고발) 1. 누구든지 범죄가 있다고 사료하는 때에는 고발할 수 있다.
---
보이스피싱 문장:
본인이 피해자이기 때문에 법적인 조치나 아무것도 들어 갈수 없어요. 이해가 되시나요? 지금 현재는 혐의자 신분이기 때문에 엠바고 사건에 대해서 유출하면 